In [1]:
import tkinter as tk
import cv2
import PIL
import PIL.Image, PIL.ImageTk
import numpy as np
from tkinter.ttk import *

In [29]:
class Runner(tk.Tk):


    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        container = tk.Frame(self)

        container.pack(side="top", fill="both", expand=True)
        self.frames = {}

        frame = StartPage(container, self)

        self.frames[StartPage] = frame
        frame.grid(row=0, column=0, sticky="nsew")
        self.show_frame(StartPage)

    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()
        
class StartPage(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        buttons_frame = tk.Frame(self)
        self.btn_load=Button(buttons_frame, text="Open\nFolder",width=10)
        self.btn_load.pack(anchor=tk.CENTER,padx =30,pady =30)
        self.btn_start=Button(buttons_frame, text="Next\nImage", width=10)
        self.btn_start.pack(anchor=tk.CENTER,padx =30,pady =30)
        self.btn_back=Button(buttons_frame, text="Previous\nImage", width=10)
        self.btn_back.pack(anchor=tk.CENTER,padx =30,pady =30)
        self.btn_stop=Button(buttons_frame, text="Save\nAnnotation", width=10)
        self.btn_stop.pack(anchor=tk.CENTER,padx =30,pady =30)
        buttons_frame.pack(pady=30,side="left",anchor="nw")
        
        
        cap_frame2 = tk.Frame(self)
        self.canvas1 = tk.Canvas(cap_frame2, width = 1000, height =800,bg="black",bd=2)
        self.canvas1.pack(padx=5,pady=5)
        cap_frame2.pack(side="left",anchor="nw",pady=30)
        
        cap_frame3 = tk.Frame(self)
        label = tk.Label(cap_frame3, text="Select Model")
        label.pack(pady = 10,padx = 20)
        self.variable = tk.StringVar(cap_frame3)
        self.models = ["FRCNN","SSD"]
        self.w = OptionMenu(cap_frame3, self.variable,*self.models)
        self.w.pack(pady = 10,padx = 20)
        cap_frame3.pack(side="left",anchor="nw",pady=30)
        
app = Runner()
app.mainloop()

l = Checkbutton(root, text=Anne, variable=['Anne', 'password1', ['friend1', 'friend2', 'friend3']]
l = Checkbutton(root, text=Bea, variable=['Bea', 'password2', ['friend1', 'friend2', 'friend3']]
l = Checkbutton(root, text=Chris, variable=['Chris', 'password1', ['friend1', 'friend2', 'friend3']]
